<a href="https://colab.research.google.com/github/vvivvi/kaggle-c1/blob/master/Kaggle_C1_text_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
import lightgbm as lgb
from sklearn.metrics import r2_score
import catboost
import gc

from catboost import CatBoostRegressor, Pool

import re
import os

for p in [np, pd, scipy, sklearn, lgb, catboost]:
    print (p.__name__, p.__version__)
    
DATA_FOLDER = 'competitive-data-science-predict-future-sales'
test_spec = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))


numpy 1.18.1
pandas 0.25.3
scipy 1.4.1
sklearn 0.22.1
lightgbm 2.3.1
catboost 0.22


In [2]:
all_data=pd.read_csv(DATA_FOLDER + '/all_data_with_lagged_targets.csv')

In [3]:
all_data.columns.values

array(['Unnamed: 0', 'Unnamed: 0.1', 'shop_id', 'item_id',
       'date_block_num', 'target', 'target_shop', 'target_item',
       'item_name', 'item_category_id',
       'item_name_category_tfidf_unigram_32',
       'item_name_category_tfidf_unigram_256',
       'item_name_category_tfidf_bigram_32',
       'item_name_category_tfidf_bigram_256',
       'item_name_category_frequent_32',
       'item_name_category_frequent_256', 'item_name_cyrillic_fraction',
       'target_category', 'target_category_tfidf_unigram_32',
       'target_category_tfidf_unigram_256',
       'target_category_tfidf_bigram_32',
       'target_category_tfidf_bigram_256', 'target_category_frequent_32',
       'target_category_frequent_256', 'target_category_within_shop',
       'target_category_tfidf_unigram_32_within_shop',
       'target_category_tfidf_unigram_256_within_shop',
       'target_category_tfidf_bigram_32_within_shop',
       'target_category_tfidf_bigram_256_within_shop',
       'target_category_fr

In [4]:
index_cols = ['shop_id','item_id','date_block_num'] # include these in all feature subsets 
target_cols = [ c for c in all_data.columns.values if re.search('^target', c) and not re.search('lag',c) and not re.search('item_within',c)]
category_cols = [ c for c in all_data.columns.values if re.search('category', c) and not re.search('target',c)]
text_feature_cols = [ c for c in all_data.columns.values if re.search('item_name_', c)]
lagged_basic_cols = [ c for c in all_data.columns.values if re.search('lag', c) and not re.search('category_.*_lag',c) and not re.search('item',c)]
lagged_within_shop_cols = [ c for c in all_data.columns.values if re.search('within_shop_lag', c) and not re.search('item',c)]
lagged_textual_cols = [ c for c in all_data.columns.values if re.search('lag', c) and re.search('category_.*_lag',c) and not re.search('within',c)] 

In [6]:
#all_data[index_cols + target_cols].to_csv(DATA_FOLDER + '/targets.csv')
df_category= all_data[index_cols + category_cols].copy()
df_category['shop_and_category'] = df_category['shop_id'].astype(str)+ '_' + df_category['item_category_id'].astype(str)
df_category.to_csv(DATA_FOLDER + '/category.csv')
#all_data[index_cols + text_feature_cols].to_csv(DATA_FOLDER + '/text_features.csv')
#all_data[index_cols + lagged_basic_cols].to_csv(DATA_FOLDER + '/lagged_basic.csv')
#all_data[index_cols + lagged_textual_cols].to_csv(DATA_FOLDER + '/lagged_textual.csv')
#all_data[index_cols + lagged_within_shop_cols].to_csv(DATA_FOLDER + '/lagged_within_shop.csv')